# Standard Library Modules

In [ ]:
import random
import time

# Third Party Modules

In [ ]:
import gurobipy as gp
import networkx as nx
from networkx.algorithms.approximation.steinertree import steiner_tree as steiner_tree_2
import scipy
from scipy.optimize import newton
from scipy.optimize import root_scalar
import sympy
from statistics import fmean as mean
from tabulate import tabulate

# Constants

## Global Constants

In [ ]:
import GlobalConstants

## Debug Level Flags

In [ ]:
import DebugConstants as db

# Problem Instances

In [ ]:
from MulticastPackingInstance import MulticastPackingInstance

# Column Generating Subproblem

## Simple 2-Approximate Steiner Tree Column Generator

In [ ]:
from Approx2MulticastPackingColumnGenerator import Approx2MulticastPackingColumnGenerator

## Steiner Tree IP Solving Column Generator

In [ ]:
from ExactMulticastPackingColumnGeneratorIP import ExactMulticastPackingColumnGeneratorIP

# Solvers

## Pure Column Generation Solver Class

In [ ]:
from PureColGenMcpSolver import PureColGenMcpSolver

## Jansen Zhang 2008 Convex MinMax Solver

In [ ]:
from JansenZhangMinMaxer import JansenZhangMinMaxer

# Testing

In [ ]:
def test_instance_generation(num_tests):
    assert not is_connected(nx.empty_graph(NUM_NODES))
    assert is_connected(get_random_connected_graph())
    G = get_random_connected_graph()
    for i in range(num_tests):
        request = MulticastRequest(MAX_MULTICAST_SIZE, G)
        source = request.source
        recipients = request.recipients
        assert request.source is not None
        assert request.recipients
        assert not request.source in request.recipients
        assert isinstance(request.recipients, set)
    instance = MulticastPackingInstance(NUM_MULTICAST_REQUESTS, MAX_MULTICAST_SIZE)
    for request in instance.requests:
        assert request.source in instance.graph
        assert request.recipients.issubset(instance.graph)

def test_LP_creation(num_tests):
    instance = MulticastPackingInstance(NUM_MULTICAST_REQUESTS, MAX_MULTICAST_SIZE)
    reduced_LP = create_LP(instance.graph, instance.requests)
    assert len(reduced_LP.getVars()) == 1
    assert len(reduced_LP.getConstrs()) == NUM_EDGES + NUM_MULTICAST_REQUESTS, "Num of constraints: {} Expected: {}".format(len(reduced_LP.getConstrs()), NUM_EDGES + NUM_MULTICAST_REQUESTS)
    
def test_subproblem(num_tests):
    instance = MulticastPackingInstance(NUM_MULTICAST_REQUESTS, MAX_MULTICAST_SIZE)
    reduced_LP = create_LP(instance.graph, instance.requests)
    column_generator = Approx2MulticastPackingColumnGenerator(instance, reduced_LP)
    
    new_trees = column_generator.generate_new_trees()
    assert (len(new_trees) == instance.num_requests)
    for tree in new_trees:
        assert cost(tree, 1) == len(tree.edges()), "cost: {} edges: {}".format(cost(tree, 1), len(tree.edges()))
           
    assert len(reduced_LP.getVars()) == 1+instance.num_requests
    
def run_tests(num_tests):
    test_instance_generation(num_tests)
    test_LP_creation(num_tests)
    test_subproblem(num_tests)

NUM_TESTS = 0
#run_tests(NUM_TESTS)

# Rounding Algorithms

## (Totally) Randomized Rounding

In [ ]:
def randRoundingHelper(fracSoln):
    randNum = random.random()
    for T in fracSoln:
        randNum -= fracSoln[T]
        if randNum < 0:
            return T
    
    
def randRounding(MCPsolver):
    intSoln = list()
    fracSoln = MCPsolver.solution[MCPsolver.iteration-1]
    for i in range(MCPsolver.instance.num_requests):
        intSoln.append(FrozenDict({randRoundingHelper(fracSoln[i]) : 1}))
        
    return tuple(intSoln)

# Run the solver

In [ ]:
instanceParameterList = [(1*GlobalConstants.NUM_NODES, 
                          1*GlobalConstants.NUM_EDGES, 
                          i*2*GlobalConstants.NUM_MULTICAST_REQUESTS, 
                          i*GlobalConstants.MAX_MULTICAST_SIZE)
                             for i in range(1,3)]
numRepititions = 2

tableHeaders = ["Solver", 
                "Block Apx Ratio", 
                "Vertices",
                "Edges",
                "Requests",
                "Group Size", 
                "Congestion", 
                "Potential",
                "Iterations",
                "Total Time",
                "Avg. Time/It"
               ]

tableRows = list()

for n,m,k,s in instanceParameterList:
    if db.DEBUG_LEVEL >= db.DEBUG_LEVEL_THEORY_0:
        print("Parameter Loop: {}, {}, {}, {}".format(n,m,k,s))
    totalIter = {(apx, solver_id): list() 
                 for apx in GlobalConstants.BLOCK_APPROX_LEVELS
                 for solver_id in GlobalConstants.SOLVER_ID_LIST}
    secsPerIt = {(apx, solver_id): list() 
                        for apx in GlobalConstants.BLOCK_APPROX_LEVELS
                        for solver_id in GlobalConstants.SOLVER_ID_LIST}
    totalTime = {(apx, solver_id): list() 
                 for apx in GlobalConstants.BLOCK_APPROX_LEVELS
                 for solver_id in GlobalConstants.SOLVER_ID_LIST}
    FinObjVal = {(apx, solver_id): list() 
                 for apx in GlobalConstants.BLOCK_APPROX_LEVELS
                 for solver_id in GlobalConstants.SOLVER_ID_LIST}
    FinPotVal = {(apx, solver_id): list() 
                 for apx in GlobalConstants.BLOCK_APPROX_LEVELS
                 for solver_id in GlobalConstants.SOLVER_ID_LIST}
    
    for i in range(numRepititions):
        if db.DEBUG_LEVEL >= db.DEBUG_LEVEL_THEORY_0:
            print("\t Instance Repitition: {}".format(i))
        instance =  MulticastPackingInstance(n,m,k,s)
        
        for apx in GlobalConstants.BLOCK_APPROX_LEVELS:
            for solver_id in GlobalConstants.SOLVER_ID_LIST:
                if solver_id == GlobalConstants.COLGEN_ID:
                    solver = PureColGenMcpSolver(instance=instance, block_approx=apx)
                elif solver_id == GlobalConstants.JZ2008_ID:
                    solver = JansenZhangMinMaxer(instance=instance, block_approx=apx)
                    
                if db.DEBUG_LEVEL >= db.DEBUG_LEVEL_THEORY_0:
                    print("\t\t Algo: {} \t Block Approx: {}".format(solver_id, apx))
                timer = list()
                while(not solver.stop_flag):
                    prev_time = time.perf_counter()
                    solver.perform_iteration()
                    new_time = time.perf_counter()
                    timer.append(new_time - prev_time)
                x = solver.solution[solver.iteration]
                totalIter[apx, solver_id].append(solver.iteration)
                secsPerIt[apx, solver_id].append(mean(timer))
                totalTime[apx, solver_id].append(sum(timer))
                FinObjVal[apx, solver_id].append(solver.lamb(x))
                FinPotVal[apx, solver_id].append(solver.phi(x, solver.t))
                
    avg_totalIter = dict()
    avg_secsPerIt = dict()
    avg_totalTime = dict()
    avg_FinObjVal = dict()
    avg_FinPotVal = dict()
    for apx in GlobalConstants.BLOCK_APPROX_LEVELS:
        for solver_id in GlobalConstants.SOLVER_ID_LIST:
            avgTotalIter = mean(totalIter[apx, solver_id])
            avgSecsPerIt = mean(secsPerIt[apx, solver_id])
            avgTotalTime = mean(totalTime[apx, solver_id])
            avgFinObjVal = mean(FinObjVal[apx, solver_id])
            avgFinPotVal = mean(FinPotVal[apx, solver_id])
            tableRows.append([solver_id, apx, n, m, k, s, 
                              avgFinObjVal, avgFinPotVal, avgTotalIter, 
                              avgTotalTime, avgSecsPerIt])
            
resultsTable = tabulate(tableRows, tableHeaders)
print(resultsTable)

# Random Stuff

In [ ]:
# for i in range(0):
#     print("BEGIN NEW TEST")
#     instance = MulticastPackingInstance(GlobalConstants.NUM_MULTICAST_REQUESTS, GlobalConstants.MAX_MULTICAST_SIZE)
#     print("Column Generation - 2-approx")
#     CGsolver = PureColGenMcpSolver(instance=instance)
#     CGtimer = list()
#     while(not CGsolver.stop_flag):
#         prev_time = time.perf_counter()
#         CGsolver.perform_iteration()
#         new_time = time.perf_counter()
#         CGtimer.append(new_time - prev_time)
#     print()
#     print("Average Time of Pure Column Generation Iteration with 2-approx block solver: {}".format(mean(CGtimer)))
#     print()
    
#     print("Column Generation - Exact")
#     CGsolver2 = PureColGenMcpSolver(instance=instance, block_approx="Exact")
#     CGtimer2 = list()
#     while(not CGsolver2.stop_flag):
#         prev_time = time.perf_counter()
#         CGsolver2.perform_iteration()
#         new_time = time.perf_counter()
#         CGtimer2.append(new_time - prev_time)
#     print()
#     print("Average Time of Pure Column Generation Iteration with an exact block solver: {}".format(mean(CGtimer2)))
#     print()
    
#     print("Jansen-Zhang Algorithm")
#     JZsolver = JansenZhangMinMaxer(instance=instance)
#     JZtimer = list()
#     while(not JZsolver.stop_flag):
#         prev_time = time.perf_counter()
#         JZsolver.perform_iteration()
#         new_time = time.perf_counter()
#         JZtimer.append(new_time - prev_time)
#     print()
#     print("Average Time of Jansen-Zhang Algorithm Iteration: {}".format(mean(JZtimer)))

In [ ]:
# x = fracSolnCG
# z = roundedCG
# for i in range(len(x)):
#     for T in z[i]:
#         print(T)
#         print(z[i][T])
#     for T in x[i]:
#         print(T)
#         print(x[i][T])

In [ ]:
# fracSolnCG = CGsolver.solution[CGsolver.iteration-1]
# fracSolnCG2 = CGsolver2.solution[CGsolver2.iteration-1]
# #fracSolnJZ = MCPsolver.solution[MCPsolver.iteration-1]
# roundedCG = randRounding(CGsolver)
# roundedCG2 = randRounding(CGsolver2)
# #roundedJZ = randRounding(JZsolver)
# print(JZsolver.lamb(roundedCG))
# print(JZsolver.lamb(roundedCG2))
# #print(JZsolver.lamb(roundedJZ))